<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2009%20-%20MultiModal_Fincance_%2B_DeepMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
from tqdm import tqdm

In [ ]:
os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
os.environ["ACTIVELOOP_TOKEN"] = "<YOUR_ACTIVELOOP_TOKEN>"

In [ ]:
# Uncomment the filenames if you don't want to process the pages using your API key.
preprocessed_texts = None#'categorized_elements.pkl'
preprocessed_graphs = None#'graphs_description.pkl'

# Extract Texts/Tables using Unstructured_IO

In [ ]:
!apt-get -qq update

In [ ]:
!apt-get -qq install poppler-utils
!apt-get -qq install tesseract-ocr

Selecting previously unselected package poppler-utils.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121688 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesse

In [ ]:
#  pdfminer.six[image]==20221105 Pillow==9.4.0

In [ ]:
!pip install -q unstructured[all-docs]==0.11.0 fastapi==0.103.2 kaleido==0.2.1 uvicorn==0.24.0.post1 typing-extensions==4.5.0 pydantic==1.10.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 27.8 MB/

In [ ]:
!wget https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update.pdf
!wget https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q2-2023-Update.pdf
!wget https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q3-2023-Update-3.pdf

--2023-12-19 17:51:22--  https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update.pdf
Resolving digitalassets.tesla.com (digitalassets.tesla.com)... 151.101.2.92, 151.101.66.92, 151.101.130.92, ...
Connecting to digitalassets.tesla.com (digitalassets.tesla.com)|151.101.2.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5935644 (5.7M) [application/pdf]
Saving to: ‘TSLA-Q1-2023-Update.pdf’

TSLA-Q1-2023-Update 100%[===================>]   5.66M  21.1MB/s    in 0.3s    

2023-12-19 17:51:22 (21.1 MB/s) - ‘TSLA-Q1-2023-Update.pdf’ saved [5935644/5935644]

--2023-12-19 17:51:22--  https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q2-2023-Update.pdf
Resolving digitalassets.tesla.com (digitalassets.tesla.com)... 151.101.2.92, 151.101.66.92, 151.101.130.92, ...
Connecting to digitalassets.tesla.com (digitalassets.tesla.com)|151.101.2.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6668267 (6.4M

In [ ]:
from pydantic import BaseModel
from typing import Any

# Define data structure
class Element(BaseModel):
    source_document: str
    type: str
    text: Any

In [ ]:
files = ['TSLA-Q1-2023-Update.pdf', 'TSLA-Q2-2023-Update.pdf', 'TSLA-Q3-2023-Update-3.pdf']

In [ ]:
if not preprocessed_texts:
  print("Processing the PDF and extracting text/tables...")
  from unstructured.partition.pdf import partition_pdf

  raw_pdf_elements = []

  for quarter in files:
    raw_pdf_elements.append( partition_pdf(
        filename=quarter,
        # Using pdf format to find embedded image blocks
        extract_images_in_pdf=False,
        # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
        # Titles are any sub-section of the document
        infer_table_structure=True,
        # Post processing to aggregate text once we have the title
        chunking_strategy="by_title",
        # Chunking params to aggregate text blocks
        # Attempt to create a new chunk 3800 chars
        # Attempt to keep chunks > 2000 chars
        # Hard max on chunks
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000
    ) )

  category_counts = {}

  for quarter in raw_pdf_elements:
    for element in quarter:
        category = str(type(element))
        if category in category_counts:
            category_counts[category] += 1
        else:
            category_counts[category] = 1

  # Unique_categories will have unique elements
  unique_categories = set(category_counts.keys())

  print( category_counts )

  # Categorize by type
  categorized_elements = []
  for idx, quarter in enumerate( raw_pdf_elements ):
    for element in quarter:
        if "unstructured.documents.elements.Table" in str(type(element)):
            categorized_elements.append(Element(type="table", text=str(element), source_document=f"2023 Quarter {idx+1} report"))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            categorized_elements.append(Element(type="text", text=str(element), source_document=f"2023 Quarter {idx+1} report"))

  with open('categorized_elements.pkl', 'wb') as handle:
    pickle.dump(categorized_elements, handle)

else:
  print("Load the pre-proocessed text/tables...")
  with open(preprocessed_texts, 'rb') as file:
      categorized_elements = pickle.load(file)

Load the pre-proocessed text/tables...


# Convert PDF to Images

In [ ]:
!pip install -q pdf2image==1.16.3

In [ ]:
from pdf2image import convert_from_path

In [ ]:
convertors = []

for quarter in files:
  convertors.append( convert_from_path(quarter) )

In [ ]:
import os
os.makedirs("./pages/Q1")
os.makedirs("./pages/Q2")
os.makedirs("./pages/Q3")

In [ ]:
for quarter, convertor in enumerate(convertors):
  for page_num, image in enumerate( convertor ):
      image.save(f"./pages/Q{quarter+1}/page-{page_num+1}.png")

In [ ]:
pages_png = []
for dir in ["./pages/Q1", "./pages/Q2", "./pages/Q3"]:
  tmp = []
  for file in os.listdir(dir):
    if file.endswith(".png"):
      file_path = os.path.join(dir, file)
      tmp.append(file_path)
  pages_png.append( tmp )

# Captioning using GPT-4V

In [ ]:
import json
import copy
import base64
import requests

headers = {
  "Content-Type": "application/json",
  "Authorization": "Bearer " + str( os.environ["OPENAI_API_KEY"] )
}

payload = {
  "model": "gpt-4-turbo",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "You are an assistant that find charts, graphs, or diagrams from an image and summarize their information. There could be multiple diagrams in one image, so explain each one of them separately. ignore tables."
        },
        {
          "type": "text",
          "text": 'The response must be a JSON in following format {"graphs": [<chart_1>, <chart_2>, <chart_3>]} where <chart_1>, <chart_2>, and <chart_3> placeholders that describe each graph found in the image. Do not append or add anything other than the JSON format response.'
        },
        {
          "type": "text",
          "text": 'If could not find a graph in the image, return an empty list JSON as follows: {"graphs": []}. Do not append or add anything other than the JSON format response. Dont use coding "```" marks or the word json.'
        },
        {
          "type": "text",
          "text": "Look at the attached image and describe all the graphs inside it in JSON format. ignore tables and be concise."
        }
      ]
    }
  ],
  "max_tokens": 1000
}

In [ ]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
if not preprocessed_graphs:
  print("Processing the PDF and extracting graph descriptions...")
  graphs_description = []
  for quarter, page_png in enumerate( pages_png ):
    tmp = []
    for page in tqdm( page_png ):
      # Getting the base64 string
      base64_image = encode_image(page)

      # Adjust Payload
      tmp_payload = copy.deepcopy(payload)
      tmp_payload['messages'][0]['content'].append({
        "type": "image_url",
        "image_url": {
          "url": f"data:image/png;base64,{base64_image}"
        }
      })

      try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=tmp_payload)
        response = response.json()
        graph_data = json.loads( response['choices'][0]['message']['content'] )['graphs']

        desc = [f"{page}\n" + '\n'.join(f"{key}: {item[key]}" for key in item.keys()) for item in graph_data]

        tmp.extend( desc )

      except:
        # Skip the page if there is an error.
        print("skipping... error in decoding.")
        continue;

    graphs_description.append( [Element(type="graph", text=str(item), source_document=f"2023 Quarter {quarter+1} report") for item in tmp] )

  with open('graphs_description.pkl', 'wb') as handle:
    pickle.dump(graphs_description, handle)

else:
  print("Load the pre-proocessed graph descriptions...")
  with open(preprocessed_graphs, 'rb') as file:
      graphs_description = pickle.load(file)

Load the pre-proocessed graph descriptions...


In [ ]:
graphs_description = [item for sublist in graphs_description for item in sublist]

In [ ]:
len( categorized_elements ), len( graphs_description )

(80, 40)

# Merge Text and Table with Images

In [ ]:
all_docs = categorized_elements + graphs_description

In [ ]:
len( all_docs )

120

# Store on DeepLake

In [ ]:
!pip install -q llama_index==0.9.8 deeplake==3.8.12 langchain==0.0.350 cohere==4.37

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
from llama_index import Document

In [ ]:
documents = [Document(text=t.text, metadata={"category": t.type, "source_document": t.source_document},) for t in categorized_elements]

In [ ]:
from llama_index.vector_stores import DeepLakeVectorStore

In [ ]:
# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path="hub://genai360/tesla_quarterly_2023-nographs",
                                   runtime={"tensor_db": True}, overwrite=False)

Your Deep Lake dataset has been successfully created!


In [ ]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Uploading data to deeplake dataset.


100%|██████████| 87/87 [00:01<00:00, 86.47it/s] 
-

Dataset(path='hub://genai360/tesla_quarterly_2023-nographs', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (87, 1)      str     None   
 metadata     json      (87, 1)      str     None   
 embedding  embedding  (87, 1536)  float32   None   
    id        text      (87, 1)      str     None   


# Inference (No Graph)

In [ ]:
from llama_index.vector_stores import DeepLakeVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index import VectorStoreIndex

In [ ]:
vector_store = DeepLakeVectorStore(dataset_path="hub://genai360/tesla_quarterly_2023-nographs", overwrite=False)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

Deep Lake Dataset in hub://genai360/tesla_quarterly_2023-nographs already exists, loading from the storage


In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query(
    "What are the trends in vehicle deliveries on quarter 3?",
)

In [ ]:
response.response

'In quarter 3, there was a decrease in Model S/X deliveries compared to the previous quarter, with a 14% decline. However, there was an increase in Model 3/Y deliveries, with a 29% growth. Overall, total deliveries in quarter 3 increased by 27% compared to the previous quarter.'

# + Deep Memory

In [ ]:
from llama_index.vector_stores import DeepLakeVectorStore

db = DeepLakeVectorStore(
    dataset_path='hub://genai360/tesla_quarterly_2023',
    runtime={"tensor_db": True},
    read_only=True,
    overwrite=False
)

Deep Lake Dataset in hub://genai360/tesla_quarterly_2023 already exists, loading from the storage


In [ ]:
# fetch dataset docs and ids if they exist (optional you can also ingest)
docs = db.vectorstore.dataset.text.data(fetch_chunks=True, aslist=True)['value']
ids = db.vectorstore.dataset.id.data(fetch_chunks=True, aslist=True)['value']
print(len(docs))

127


In [ ]:
import json
from openai import OpenAI

client = OpenAI()
# Set the function JSON Schema for openai function calling feature
tools = [
    {
        "type": "function",
        "function": {
            "name": "create_question_from_text",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "Question created from the given text",
                    },
                },
                "required": ["question"],
            },
            "description": "Create question from a given text.",
        },
    }
]


def generate_question(tools, text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            tools=tools,
            tool_choice={
                "type": "function",
                "function": {"name": "create_question_from_text"},
            },
            messages=[
                {"role": "system", "content": "You are a world class expert for generating questions based on provided context. \
                        You make sure the question can be answered by the text."},
                {
                    "role": "user",
                    "content": text,
                },
            ],
        )

        json_response = response.choices[0].message.tool_calls[0].function.arguments
        parsed_response = json.loads(json_response)
        question_string = parsed_response["question"]
        return question_string
    except:
        question_string = "No question generated"
        return question_string

In [ ]:
import random
from tqdm import tqdm

def generate_queries(docs: list[str], ids: list[str], n: int):

    questions = []
    relevances = []
    pbar = tqdm(total=n)
    while len(questions) < n:
        # 1. randomly draw a piece of text and relevance id
        r = random.randint(0, len(docs)-1)
        text, label = docs[r], ids[r]

        # 2. generate queries and assign and relevance id
        generated_qs = [generate_question(tools, text)]
        if generated_qs == ["No question generated"]:
            continue

        questions.extend(generated_qs)
        relevances.extend([[(label, 1)] for _ in generated_qs])
        pbar.update(len(generated_qs))

    return questions[:n], relevances[:n]

questions, relevances = generate_queries(docs, ids, n=20)

100%|██████████| 20/20 [00:19<00:00,  1.02it/s]


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

job_id = db.vectorstore.deep_memory.train(
    queries=questions,
    relevance=relevances,
    embedding_function=embeddings.embed_documents,
)

Starting DeepMemory training job
Your Deep Lake dataset has been successfully created!


Preparing training data for deepmemory:


Creating 20 embeddings in 1 batches of size 20:: 100%|██████████| 1/1 [00:03<00:00,  3.23s/it]


DeepMemory training job started. Job ID: 6581e3056a1162b64061a9a4


In [ ]:
db.vectorstore.dataset.embedding.info

{'deepmemory': {'6581e3056a1162b64061a9a4_0.npy': {'base_recall@10': 0.25, 'deep_memory_version': '0.2', 'delta': 0.25, 'job_id': '6581e3056a1162b64061a9a4_0', 'model_type': 'npy', 'recall@10': 0.5}, 'model.npy': {'base_recall@10': 0.25, 'deep_memory_version': '0.2', 'delta': 0.25, 'job_id': '6581e3056a1162b64061a9a4_0', 'model_type': 'npy', 'recall@10': 0.5}}}

In [ ]:
db.vectorstore.deep_memory.status('6581e3056a1162b64061a9a4')

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/genai360/tesla_quarterly_2023


TypeError: ignored

## Inference (With Graph and Deep Memory)

In [ ]:
vector_store = DeepLakeVectorStore(dataset_path="hub://genai360/tesla_quarterly_2023", overwrite=False)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

Deep Lake Dataset in hub://genai360/tesla_quarterly_2023 already exists, loading from the storage


In [ ]:
query_engine = index.as_query_engine(vector_store_kwargs={"deep_memory": True})

In [ ]:
response = query_engine.query(
    "What are the trends in vehicle deliveries on quarter 3 report?",
)

In [ ]:
response.response

'The trends in vehicle deliveries on the Quarter 3 report show an increasing trend over the quarters.'

# Environment Snapshot

In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aioboto3                         12.0.0
aiobotocore                      2.7.0
aiohttp                          3.8.6
aioitertools                     0.11.0
aiosignal                        1.3.1
aiostream                        0.5.2
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
antlr4-python3-runtime           4.9.3
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
au